<a href="https://colab.research.google.com/github/HackElite-FYP/Legal-Research-Platform-Core/blob/feature%2Fsummarization/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# install dependencies
# !python.exe -m pip install --upgrade pip

# %pip install pandas
# %pip install nltk
# %pip install numpy --only-binary :all:
# %pip install transformers sumy sentencepiece
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# %pip install notebook ipywidgets --upgrade
# %pip install language_tool_python

!pip install pandas numpy sumy
!pip install transformers nltk sentencepiece
!pip install torch
!pip install language_tool_python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# init variables
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # This might not be necessary in Colab and can sometimes cause issues.
# In Colab, it's generally better to let the environment manage CUDA.

PROCESSING_FILE_PATH = '/content/drive/MyDrive/FYP/json/cases_2024.json'
PROCESSING_CASE_INDEX = 0
SUMMARY_MODELS = ['facebook/bart-large-cnn', 'google/pegasus-xsum', 't5-base', 'allenai/led-base-16384']

In [9]:
# load to dataframes
import json
import pandas as pd

with open(PROCESSING_FILE_PATH, 'r', encoding='utf-8') as f:
    cases_data = json.load(f)

cases_df = pd.DataFrame(cases_data)
print(cases_df.head())

                                     id     type amendmentTo  \
0  a62196ef-f9a5-4ee5-98f0-7c2642a53d84      act               
1  d1c8e669-eeb8-4b95-8d69-967129049b2d      act               
2  61eace73-f7ef-4254-843c-4d223e1577e1  unknown               
3  1bff7aa5-709b-4251-921c-705a6c9cccd5     case               
4  f0d95d5d-0594-4a5f-a5bc-5ccac65b20a2      act               

                                       filename primaryLang  \
0           cpa_0132_23_final_judgement_pdf.pdf          en   
1  court_of_appeal_judgment_hcc_0184_17_pdf.pdf          en   
2                        ca_writ_170_22_pdf.pdf          en   
3              wrt_0201_21_31_01_2024_1_pdf.pdf          en   
4        ca_phc_0066_12_final_judgement_pdf.pdf          en   

                                               title  \
0  The petitioner is seeking to challenge the ord...   
1  CA/HCC 184/2017 IN THE COURT OF APPEAL OF THE ...   
2                                           Untitled   
3           

In [13]:
# sentence tokenize
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download('punkt_tab', 'resources/dependencies/nltk') # This path is for local download.
nltk.download('punkt_tab') # Download to the default nltk data path in Colab

cases_df['sentences'] = cases_df['cleanedText'].apply(sent_tokenize)
print(cases_df['sentences'])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


0      [The Attorney General,\nAttorney General’s Dep...
1      [ read with\nArticle 138 of the Constitution o...
2      [IN THE COURT OF APPEAL OF THE DEMOCRATIC SOCI...
3      [WICKUM A. KALUARACHCHI, J., The Petitioner Co...
4      [., Court of Appeal No: Wanninayaka Mudiyansel...
                             ...                        
525    [CP(PHC)APN 144/2022\nIN THE COURT OF APPEAL O...
526    [\n(‘TEWA’)., In the said Order ‘P9’ the 1st R...
527    [C.A., WRIT 88-2019\nIN THE COURT OF APPEAL OF...
528    [\nCourt of Appeal Case No., 1., Wadduwage Ruw...
529    [11., Prof. Mohan de Silva\nFormer Chairman., ...
Name: sentences, Length: 530, dtype: object


In [15]:
# ============== extractive summarization (unsupervised) ==============
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

text = cases_df.loc[PROCESSING_CASE_INDEX, 'cleanedText']
parser = PlaintextParser.from_string(text, Tokenizer("english"))
summarizer = LexRankSummarizer()

summary = summarizer(parser.document, sentences_count=5)
for sentence in summary:
    print(sentence)

The accused has appeared before the High Court on notice on 16-09-2020, and after serving the indictment and other relevant documents on the accused, the learned High Court Judge of Kandy has released the accused on bail.
Page 3 of 11 When the above application was made, the learned High Court Judge has made the impugned order sought to be challenged before this Court on behalf of the accused.
Therefore, it is quite obvious that although section 263 of the Code of Criminal Procedure Act provides for the remanding of a person pending further trial, the provisions of the Bail Act shall prevail over the said provision when it comes to the question of bail or cancellation of bail for that matter.
The Attorney General (2003) 2 SLR 39 it was held: “In terms of the mandatory requirements of section 14(1) such cancellation could have been done only on:- Page 8 of 11 (i) An application being made by a police officer; (ii) Hearing the accused appellant personally or through his attorney-at-law; 

In [ ]:
# ============== Extractive Summarization (Hierarchical Approach) ==============
from transformers import pipeline, AutoTokenizer
import torch

# Chunking function with overlap to maintain context
def chunk_text(text, tokenizer, max_tokens=900, overlap_tokens=100):
    tokens = tokenizer.encode(text)
    total_tokens = len(tokens)
    chunks = []
    start = 0
    while start < total_tokens:
        end = min(start + max_tokens, total_tokens)
        chunk = tokenizer.decode(tokens[start:end], skip_special_tokens=True)
        chunks.append(chunk)
        if end == total_tokens:
            break
        start += (max_tokens - overlap_tokens)
    return chunks

# Initialize summarizer (GPU or CPU)
import torch
device = 0 if torch.cuda.is_available() else -1

model_name = SUMMARY_MODELS[0]
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model_name, device=device)
text = cases_df.loc[PROCESSING_CASE_INDEX, 'text']

# Hierarchical summarization function with dynamic max_length
def hierarchical_summary(text, summarizer, tokenizer,
                         max_chunk_tokens=900, overlap_tokens=100,
                         chunk_summary_max_len=150, final_summary_max_len=200):

    # Chunk the original document
    chunks = chunk_text(text, tokenizer, max_tokens=max_chunk_tokens, overlap_tokens=overlap_tokens)

    print(f"Number of chunks: {len(chunks)}")

    # Summarize each chunk individually
    intermediate_summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)}...")
        chunk_len = len(tokenizer.encode(chunk))
        adjusted_max_len = min(chunk_summary_max_len, max(30, int(chunk_len * 0.5)))

        summary = summarizer(
            chunk,
            max_length=adjusted_max_len,
            min_length=min(20, adjusted_max_len//2),
            do_sample=False
        )[0]['summary_text']

        intermediate_summaries.append(summary)

    # Combine intermediate summaries
    combined_summary_text = " ".join(intermediate_summaries)

    # Generate final summary from intermediate summaries
    print("Generating final summary...")
    final_summary = summarizer(
        combined_summary_text,
        max_length=final_summary_max_len,
        min_length=100,
        do_sample=False
    )[0]['summary_text']

    return final_summary

# Example Usage
final_summary = hierarchical_summary(
    text, summarizer, tokenizer,
    max_chunk_tokens=900,
    overlap_tokens=100,
    chunk_summary_max_len=150,
    final_summary_max_len=200
)

print("Final Summary:")
print(final_summary)

Device set to use cuda:0


Number of chunks: 7
Summarizing chunk 1/7...
Summarizing chunk 2/7...
Summarizing chunk 3/7...
Summarizing chunk 4/7...
Summarizing chunk 5/7...
Summarizing chunk 6/7...
Summarizing chunk 7/7...
Generating final summary...
Final Summary:
Petitioner is seeking to challenge the Order Made by the Learned High. The Attorney General, the Attorney General's department, is the respondent. The Accused has been charged with Grave Sexual Abuse o f a minor. He has been remanded on bail until the end of the trial. I am of the view that the learned high judge WAS MISDIRECTED. The onely Assumtion That CAN BE MADE is that the Remanding of The Accuses For A Period of 3 months HAD BEEN DONE AS A PUNITIVE measure. The order mode by this court previcly on 15 -12-2023, to release the Accused.


In [ ]:
from transformers import pipeline
import torch

# Use GPU if available
device = 0 if torch.cuda.is_available() else -1

# Initialize refinement pipeline
refiner = pipeline("text2text-generation", model="google/flan-t5-base", device=device)

def refine_legal_summary(summary_text):
    prompt = (
        "Refine the following legal summary. Correct grammar, spelling, punctuation, "
        "remove repetition, and ensure clarity without changing any legal meaning:\n\n"
        f"{summary_text}"
    )

    refined_output = refiner(prompt, max_length=256, do_sample=False)
    refined_summary = refined_output[0]['generated_text']

    return refined_summary.strip()

# Example usage:
clean_summary = refine_legal_summary(final_summary)
print("Refined Final Summary:")
print(clean_summary)

Device set to use cuda:0


Refined Final Summary:
Petitioner is seeking to challenge the Order Made by the Learned High. The Attorney General, the Attorney General's department, is the respondent. The Accused has been charged with Grave Sexual Abuse o f a minor. He has been remanded on bail until the end of the trial. I am of the view that the learned high judge WAS MISDIRECTED. The onely Assumtion That CAN BE MADE is that the Remanding of The Accuses For A Period of 3 months HAD BEEN DONE AS A PUNITIVE measure. The order mode by this court previcly on 15 -12-2023, to release the Accused.
